In [76]:
from pathlib import Path
import pandas as pd
import numpy as np

RAW = Path("../data/raw")
OUT = Path("../data/processed")
#OUT.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 200)


In [77]:
regular_path = RAW / "MRegularSeasonCompactResults.csv"
tourney_path = RAW / "MNCAATourneyCompactResults.csv"
teams_path   = RAW / "MTeams.csv"
seeds_path   = RAW / "MNCAATourneySeeds.csv"

regular = pd.read_csv(regular_path)
tourney = pd.read_csv(tourney_path)
teams   = pd.read_csv(teams_path)

seeds = None
if seeds_path.exists():
    seeds = pd.read_csv(seeds_path)

print("regular:", regular.shape)
print("tourney:", tourney.shape)
print("teams:", teams.shape)
print("seeds:", None if seeds is None else seeds.shape)


regular: (192930, 8)
tourney: (2518, 8)
teams: (380, 4)
seeds: (2626, 3)


In [78]:
def make_team_game_table(games: pd.DataFrame) -> pd.DataFrame:
    """
    Converts compact results into a per-team-per-game table.
    Expected columns (typical):
      Season, DayNum, WTeamID, LTeamID, WScore, LScore
    Returns columns:
      Season, DayNum, TeamID, OppTeamID, is_win, PF, PA, margin
    """
    required = {"Season", "WTeamID", "LTeamID", "WScore", "LScore"}
    missing = required - set(games.columns)
    if missing:
        raise ValueError(f"Missing columns in results file: {missing}")

    # winner perspective
    w = games[["Season"] + (["DayNum"] if "DayNum" in games.columns else []) +
              ["WTeamID", "LTeamID", "WScore", "LScore"]].copy()
    w = w.rename(columns={
        "WTeamID": "TeamID",
        "LTeamID": "OppTeamID",
        "WScore": "PF",
        "LScore": "PA"
    })
    w["is_win"] = 1

    # loser perspective
    l = games[["Season"] + (["DayNum"] if "DayNum" in games.columns else []) +
              ["WTeamID", "LTeamID", "WScore", "LScore"]].copy()
    l = l.rename(columns={
        "LTeamID": "TeamID",
        "WTeamID": "OppTeamID",
        "LScore": "PF",
        "WScore": "PA"
    })
    l["is_win"] = 0

    tg = pd.concat([w, l], ignore_index=True)
    tg["margin"] = tg["PF"] - tg["PA"]
    return tg

regular_team_games = make_team_game_table(regular)
tourney_team_games = make_team_game_table(tourney)

regular_team_games.head()


,Season,DayNum,TeamID,OppTeamID,PF,PA,is_win,margin
0,1985,20,1228,1328,81,64,1,17
1,1985,25,1106,1354,77,70,1,7
2,1985,25,1112,1223,63,56,1,7
3,1985,25,1165,1432,70,54,1,16
4,1985,25,1192,1447,86,74,1,12


In [79]:
def build_team_season_features(team_games: pd.DataFrame) -> pd.DataFrame:
    feats = team_games.groupby(["Season", "TeamID"], as_index=False).agg(
        games=("is_win", "size"),
        wins=("is_win", "sum"),
        avg_pf=("PF", "mean"),
        avg_pa=("PA", "mean"),
        avg_margin=("margin", "mean"),
        std_margin=("margin", "std"),
    )
    feats["win_pct"] = feats["wins"] / feats["games"]
    feats["std_margin"] = feats["std_margin"].fillna(0)
    return feats

regular_team_season = build_team_season_features(regular_team_games)
regular_team_season.to_csv(
    OUT / "regular_team_season_features.csv",
    index=False
)

regular_team_season.head()


,Season,TeamID,games,wins,avg_pf,avg_pa,avg_margin,std_margin,win_pct
0,1985,1102,24,5,63.083333,68.875000,-5.791667,12.714180,0.208333
1,1985,1103,23,9,61.043478,64.086957,-3.043478,10.674857,0.391304
2,1985,1104,30,21,68.500000,60.700000,7.800000,10.905234,0.700000
3,1985,1106,24,10,71.625000,75.416667,-3.791667,15.353740,0.416667
4,1985,1108,25,19,83.000000,75.040000,7.960000,14.426480,0.760000


In [80]:
#Build seed table
def clean_seeds(seeds_df):
    s = seeds_df.copy()
    s["SeedNum"] = s["Seed"].astype(str).str.extract(r"(\d+)").astype(int)
    return s[["Season", "TeamID", "Seed", "SeedNum"]]

tourney_seeds = clean_seeds(seeds)

print("Built seeds table:", tourney_seeds.shape)
tourney_seeds.head(10)

Built seeds table: (2626, 4)


,Season,TeamID,Seed,SeedNum
0,1985,1207,W01,1
1,1985,1210,W02,2
2,1985,1228,W03,3
3,1985,1260,W04,4
4,1985,1374,W05,5
5,1985,1208,W06,6
6,1985,1393,W07,7
7,1985,1396,W08,8
8,1985,1439,W09,9
9,1985,1177,W10,10


In [81]:
#Check
regular_team_season.query("Season == 2024").sort_values("win_pct", ascending=False).head(10)

,Season,TeamID,games,wins,avg_pf,avg_pa,avg_margin,std_margin,win_pct
12718,2024,1163,34,31,81.470588,64.411765,17.058824,15.334535,0.911765
12794,2024,1241,33,30,82.969697,69.484848,13.484848,11.613896,0.909091
12823,2024,1270,29,26,78.448276,64.034483,14.413793,12.439905,0.896552
12775,2024,1222,34,30,73.029412,56.970588,16.058824,16.596716,0.882353
12895,2024,1345,33,29,83.393939,70.151515,13.242424,14.106803,0.878788
12768,2024,1213,32,28,78.406250,66.687500,11.718750,10.952196,0.875000
12893,2024,1343,26,22,76.230769,67.500000,8.730769,10.436696,0.846154
12909,2024,1359,32,27,84.343750,74.375000,9.968750,17.642479,0.843750
12734,2024,1179,33,27,80.272727,70.606061,9.666667,14.540174,0.818182
12785,2024,1232,33,27,84.242424,72.363636,11.878788,15.253601,0.818182
